In [1]:
# post_processing.ipynb

# [1] Initialization Cell
# -----------------------
import pandas as pd
import numpy as np
from pathlib import Path
import json
import traceback
from datetime import datetime
from tqdm.notebook import tqdm

print("✅ Libraries imported")

✅ Libraries imported


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# !rm data/csv/*.csv
# !rm data/post_csvs/*.csv
# !rm data/csv_dup_rows/*.csv
# !rm data/global_posts.csv
# !rm data/accounts_analysis.csv


# 1. Convert NDjsons to CSVs

In [4]:
import pandas as pd

accounts_df = pd.read_csv("data/accounts.csv")

#accounts_df = accounts_df.head(5) #rm
accounts_df = accounts_df.drop(index=1) #rm


In [5]:
accounts_df.insert(1, 'IG_handle', accounts_df['Account-Link'].str.replace("https://www.instagram.com/","").str.replace("/",""))

In [6]:
accounts_df

,Account-Link,IG_handle,IG_Followers_manual,scrape_status,Export_Path,CSV_Path,last_scraped,ZS_count,IG_count,IG_Followers,IG_Followed,Scrape_Start,Scrape_End,Notes,Name,Lebensdaten,Fraktion oder Gruppe (Partei),Land,Listenplatz,Wahlkreis,Erststimmenanteil,Beruflicher Hintergrund,MdB seit,Bemerkungen,Unnamed: 12,Arbeitseinteilung
0,https://www.instagram.com/bundeskanzler/,bundeskanzler,2000000.0,completed,data/exports/20250215_190918_bundeskanzler.ndjson,NaN,2025-02-15 19:09:20.938660,1200,1200,2000000,109,2025-02-15 19:04:35,2025-02-15 19:09:18,NaN,Olaf Scholz als Bundeskanzler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.instagram.com/abaerbock/,abaerbock,611000.0,completed,data/exports/20250215_191752_abaerbock.ndjson,NaN,2025-02-15 19:17:54.514807,964,964,637000,419,2025-02-15 19:13:40,2025-02-15 19:17:52,NaN,Annalena Baerbock,1980.0,Grüne,Brandenburg,Platz 1,Potsdam – Potsdam-Mittelmark II – Teltow-Flämi...,"18,8 %",Völkerrechtlerin,2013,"Bundesministerin des Auswärtigen,",Parteivorsitzende,(bis Februar 2022)
3,https://www.instagram.com/minister.habeck/,minister.habeck,560080.0,completed,data/exports/20250215_192331_minister.habeck.n...,NaN,2025-02-15 19:23:34.056458,1510,1510,575000,34,2025-02-15 19:17:56,2025-02-15 19:23:31,NaN,Robert Habeck,1969.0,Grüne,Schleswig-Holstein,Platz 2,Flensburg – Schleswig,"28,1 %",Politiker,2021,Bundesminister für Wirtschaft und Klimaschutz ...,NaN,NaN
4,https://www.instagram.com/dorobaer/,dorobaer,502000.0,done,data/exports/20250219_022801_dorobaer.ndjson,NaN,2025-02-19T02:28:03.974044,2717,2720,51000,5630,2025-02-15 19:23:37,2025-02-19 02:28:01,Accepting partial capture (2709/2713) after 5 ...,Dorothee Bär,1978.0,CDU/CSU (CSU),Bayern,Platz 2,Bad Kissingen,"39,1 %",Politologin,2002,NaN,NaN,NaN
5,https://www.instagram.com/chrupallatino/,chrupallatino,393000.0,completed,data/exports/20250215_193744_chrupallatino.ndjson,NaN,2025-02-15 19:37:46.762136,700,700,52200,166,2025-02-15 19:35:09,2025-02-15 19:37:44,NaN,Tino Chrupalla,1975.0,AfD,Sachsen,Platz 1,Görlitz,"35,8 %",Malermeister,2017,"Fraktionsvorsitzender,",Parteivorsitzender,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,NaN,NaN,NaN,pending,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,(SSW),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
746,NaN,NaN,NaN,pending,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,Max Straubinger,1954.0,CDU/CSU (CSU),Bayern,–,Rottal-Inn,"31,5 %",Versicherungskaufmann,1994,NaN,NaN,NaN
747,NaN,NaN,NaN,pending,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,Marco Wanderwitz,1975.0,CDU/CSU (CDU),Sachsen,Platz 1,Chemnitzer Umland – Erzgebirgskreis II,"23,7 %",Rechtsanwalt,2002,NaN,NaN,NaN
748,NaN,NaN,NaN,pending,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(ab 14. Juli 2022),NaN,NaN


In [7]:
!pip install tqdm

from tqdm.auto import tqdm

/Users/leobru/anaconda3/envs/bundesposts_scraping/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Michael Achmann. (2023). michaelachmann/social-media-lab: 06.11.2023 (v0.0.3). Zenodo. https://doi.org/10.5281/zenodo.8199901

import pandas as pd
import json
#@title Import Zeeschuimer Data
#@markdown **Keep 4CAT as format to download images within this notebook.**  **Formats:** *MA* corresponds to CrowdTangle and instaloader Notebooks. Compatible with future notebooks for data annotation. *4CAT* corresponds to 4CAT CSV format.  *TODO: Import multiple images.*  This importer is based on 4CAT code, published under Mozilla Pulice License 2.0 at https://github.com/digitalmethodsinitiative/4cat/tree/master/datasources/instagram

from tqdm.auto import tqdm
#from datetime import datetime

######### LB
def convert_single_ndjson_to_csv(input_ndjson_path, output_csv_path):
######### LB

    import_format = "4CAT" # @param ["MA", "4CAT"]

    # import_filename = 'data/exports/20250215_192331_minister.habeck.ndjson'  # @param {type: "string"}
    # export_filename = 'data/csv/test_habeck.csv'  # @param {type: "string"}
    
    import_filename = input_ndjson_path
    export_filename = output_csv_path

    """
    Import scraped Instagram data

    This code has been taken from 4CAT, available under Mozilla Public License 2.0 at https://github.com/digitalmethodsinitiative/4cat/tree/master/datasources/instagram
    """
    from datetime import datetime
    import re

    # some magic numbers instagram uses
    MEDIA_TYPE_PHOTO = 1
    MEDIA_TYPE_VIDEO = 2
    MEDIA_TYPE_CAROUSEL = 8

    def parse_graph_item(node):
        """
        Parse Instagram post in Graph format

        :param node:  Data as received from Instagram
        :return dict:  Mapped item
        """
        try:
            caption = node["edge_media_to_caption"]["edges"][0]["node"]["text"]
        except IndexError:
            caption = ""

        num_media = 1 if node["__typename"] != "GraphSidecar" else len(node["edge_sidecar_to_children"]["edges"])

        # get media url
        # for carousels, get the first media item, for videos, get the video
        # url, for photos, get the highest resolution

        media_nodes = []

        if node["__typename"] == "GraphSidecar":
            for media_node in node["edge_sidecar_to_children"]["edges"]:
                media_nodes.append(media_node["node"])
        else:
            media_node.apend(node)

        mapped_items = []
        for media_node in media_nodes:
            if media_node["__typename"] == "GraphVideo":
                media_url = media_node["video_url"]
            elif media_node["__typename"] == "GraphImage":
                resources = media_node.get("display_resources", media_node.get("thumbnail_resources"))
                try:
                    media_url = resources.pop()["src"]
                except AttributeError:
                    media_url = media_node.get("display_url", "")
            else:
                media_url = media_node["display_url"]

            # type, 'mixed' means carousel with video and photo
            type_map = {"GraphSidecar": "photo", "GraphVideo": "video"}
            if node["__typename"] != "GraphSidecar":
                media_type = type_map.get(node["__typename"], "unknown")
            else:
                media_types = set([s["node"]["__typename"] for s in node["edge_sidecar_to_children"]["edges"]])
                media_type = "mixed" if len(media_types) > 1 else type_map.get(media_types.pop(), "unknown")

            location = {"name": "", "latlong": "", "city": ""}
            # location has 'id', 'has_public_page', 'name', and 'slug' keys in tested examples; no lat long or "city" though name seems
            if node.get("location"):
                location["name"] = node["location"].get("name")
                # Leaving this though it does not appear to be used in this type; maybe we'll be surprised in the future...
                location["latlong"] = str(node["location"]["lat"]) + "," + str(node["location"]["lng"]) if node[
                    "location"].get("lat") else ""
                location["city"] = node["location"].get("city")


            if num_media > 1:
                media_filename = f"{node['code']}_{i}"
            else:
                media_filename = f"{node['code']}"

            mapped_item = {
                "id": node["shortcode"],
                "thread_id": node["shortcode"],
                "parent_id": node["shortcode"],
                "body": caption,
                "author": node["owner"]["username"],
                "timestamp": datetime.fromtimestamp(node["taken_at_timestamp"]).strftime("%Y-%m-%d %H:%M:%S"),
                "author_fullname": node["owner"].get("full_name", ""),
                "author_avatar_url": node["owner"].get("profile_pic_url", ""),
                "type": media_type,
                "url": "https://www.instagram.com/p/" + node["shortcode"],
                "image_url": node["display_url"],
                "media_url": media_url,
                "hashtags": ",".join(re.findall(r"#([^\s!@#$%ˆ&*()_+{}:\"|<>?;'\,./`~']+)", caption)),
                # "usertags": ",".join(
                #     [u["node"]["user"]["username"] for u in node["edge_media_to_tagged_user"]["edges"]]),
                "num_likes": node["edge_media_preview_like"]["count"],
                "num_comments": node.get("edge_media_preview_comment", {}).get("count", 0),
                "num_media": num_media,
                "location_name": location["name"],
                "location_latlong": location["latlong"],
                "location_city": location["city"],
                "unix_timestamp": node["taken_at_timestamp"],
                "media_n": i,
                "media_filename": media_filename
            }
            mapped_items.append(mapped_item)

        return mapped_items

    @staticmethod
    def parse_itemlist_item(node):
        """
        Parse Instagram post in 'item list' format

        :param node:  Data as received from Instagram
        :return dict:  Mapped item
        """
        num_media = 1 if node["media_type"] != MEDIA_TYPE_CAROUSEL else len(node["carousel_media"])
        caption = "" if not node.get("caption") else node["caption"]["text"]

        # get media url
        # for carousels, get the first media item, for videos, get the video
        # url, for photos, get the highest resolution

        media_nodes = []

        if node["media_type"] == MEDIA_TYPE_CAROUSEL:
            media_nodes.extend(node["carousel_media"])
        else:
            media_nodes.append(node)


        mapped_items = []

        for i, media_node in enumerate(media_nodes):
            if media_node["media_type"] == MEDIA_TYPE_VIDEO:
                media_url = media_node["video_versions"][0]["url"]
                if "image_versions2" in media_node:
                    display_url = media_node["image_versions2"]["candidates"][0]["url"]
                else:
                    # no image links at all :-/
                    # video is all we have
                    display_url = media_node["video_versions"][0]["url"]
            elif media_node["media_type"] == MEDIA_TYPE_PHOTO:
                media_url = media_node["image_versions2"]["candidates"][0]["url"]
                display_url = media_url
            else:
                media_url = ""
                display_url = ""

            # type, 'mixed' means carousel with video and photo
            type_map = {MEDIA_TYPE_PHOTO: "photo", MEDIA_TYPE_VIDEO: "video"}
            if node["media_type"] != MEDIA_TYPE_CAROUSEL:
                media_type = type_map.get(node["media_type"], "unknown")
            else:
                media_types = set([s["media_type"] for s in node["carousel_media"]])
                media_type = "mixed" if len(media_types) > 1 else type_map.get(media_types.pop(), "unknown")

            if "comment_count" in node:
                num_comments = node["comment_count"]
            elif "comments" in node and type(node["comments"]) is list:
                num_comments = len(node["comments"])
            else:
                num_comments = -1

            location = {"name": "", "latlong": "", "city": ""}
            if node.get("location"):
                location["name"] = node["location"].get("name")
                location["latlong"] = str(node["location"]["lat"]) + "," + str(node["location"]["lng"]) if node[
                    "location"].get("lat") else ""
                location["city"] = node["location"].get("city")

            if num_media > 1:
                media_filename = f"{node['code']}_{i}"
            else:
                media_filename = f"{node['code']}"

            mapped_item = {
                "id": node["code"],
                "thread_id": node["code"],
                "parent_id": node["code"],
                "body": caption,
                "author": node["user"]["username"],
                "author_fullname": node["user"]["full_name"],
                "author_avatar_url": node["user"]["profile_pic_url"],
                "timestamp": datetime.fromtimestamp(node["taken_at"]).strftime("%Y-%m-%d %H:%M:%S"),
                "type": media_type,
                "url": "https://www.instagram.com/p/" + node["code"],
                "image_url": display_url,
                "media_url": media_url,
                "hashtags": ",".join(re.findall(r"#([^\s!@#$%ˆ&*()_+{}:\"|<>?;'\,./`~']+)", caption)),
                # "usertags": ",".join(
                #     [u["node"]["user"]["username"] for u in node["edge_media_to_tagged_user"]["edges"]]),
                "num_likes": node["like_count"],
                "num_comments": num_comments,
                "num_media": num_media,
                "location_name": location["name"],
                "location_latlong": location["latlong"],
                "location_city": location["city"],
                "unix_timestamp": node["taken_at"],
                "media_n": i,
                "media_filename": media_filename
            }
            mapped_items.append(mapped_item)

        return mapped_items

    def map_item(item):
        """
        Map Instagram item

        Instagram importing is a little bit roundabout since we can expect
        input in two separate and not completely overlapping formats - an "edge
        list" or an "item list", and posts are structured differently between
        those, and do not contain the same data. So we find a middle ground
        here... each format has its own handler function

        :param dict item:  Item to map
        :return:  Mapped item
        """
        link = item.get("link", "")
        if (item.get("product_type", "") == "ad") or \
                (link and link.startswith("https://www.facebook.com/ads/ig_redirect")):

            return None

        is_graph_response = "__typename" in item and item["__typename"] not in ("XDTMediaDict",)

        if is_graph_response:
            return parse_graph_item(item)
        else:
            return parse_itemlist_item(item)



    ##########
    # CODE MA
    ##########

    data = []

    # List to store all the dictionaries
    data_list = []

    # Read the ndjson file line by line
    with open(import_filename, 'r') as file:
        for line in file:
            # Parse the JSON line and extract the 'data' field
            json_line = json.loads(line)
            data_field = json_line['data']

            # Add the 'data' field to your list
            data_list.append(data_field)


    # Next, we convert the data into a table
    for element in tqdm(data_list):
        item = map_item(element)

        if item:
            if import_format == "MA":
                data.append({
                    'shortcode': item.get("id", ""),
                    'username': item.get("author", ""),
                    'timestamp': item.get("unix_timestamp", None),
                    'caption': item.get("body", ""),
                    'location': item.get("location_name", None),
                })

            else:
                data.extend(item)

    posts_df = pd.DataFrame(data)
    posts_df.to_csv(export_filename)

    print(f"Imported Zeeschuimer Data. Saved export to {export_filename}")



In [9]:
def convert_all_ndjson_to_csv(accounts_df):
   
   
  for idx, row in accounts_df.iterrows():
    try:
      # Check for required fields
      if pd.isna(row.get("Export_Path")) or pd.isna(row.get("IG_handle")):
          print(f"⚠️ Skipping row {idx}: Missing Export_Path or IG_handle")
          continue

      import_filename = row["Export_Path"]

      handle = row["IG_handle"]

      # Verify source file exists
      if not Path(import_filename).exists():
          print(f"⚠️ File not found: {import_filename} - Skipping")
          continue

      timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

      export_filename = f"data/csv_dup_rows/{timestamp}_{handle}.csv"

      # Process file with error handling
      try:
          convert_single_ndjson_to_csv(import_filename, export_filename)
          accounts_df.at[idx, "CSV_Path"] = export_filename
          print(f"✅ Success: {export_filename}")
      except Exception as e:
          print(f"🔥 Error processing {import_filename}: {str(e)}")
          traceback.print_exc()


    except KeyError as ke:
        print(f"❌ Missing column in dataframe: {str(ke)}")
    except Exception as e:
        print(f"🔥 Unexpected error with row {idx}: {str(e)}")

    # Save only if dataframe was modified        
    if "CSV_Path" in accounts_df.columns:
        accounts_df.to_csv("data/csv_dup_rows/accounts_dup_rows.csv", index=False)
        print("💾 Saved updated accounts list")
    else:
        print("🔄 No changes to save")



  

In [10]:
convert_all_ndjson_to_csv(accounts_df)

100%|██████████| 1200/1200 [00:00<00:00, 18333.48it/s]


Imported Zeeschuimer Data. Saved export to data/csv_dup_rows/20250222_050038_bundeskanzler.csv
✅ Success: data/csv_dup_rows/20250222_050038_bundeskanzler.csv
💾 Saved updated accounts list


100%|██████████| 964/964 [00:00<00:00, 28025.79it/s]


Imported Zeeschuimer Data. Saved export to data/csv_dup_rows/20250222_050038_abaerbock.csv
✅ Success: data/csv_dup_rows/20250222_050038_abaerbock.csv
💾 Saved updated accounts list


100%|██████████| 1510/1510 [00:00<00:00, 51363.68it/s]


...

Imported Zeeschuimer Data. Saved export to data/csv_dup_rows/20250222_051443_wiebkepapenbrock.csv
✅ Success: data/csv_dup_rows/20250222_051443_wiebkepapenbrock.csv
💾 Saved updated accounts list


100%|██████████| 32/32 [00:00<00:00, 24767.99it/s]


Imported Zeeschuimer Data. Saved export to data/csv_dup_rows/20250222_051443_markus.herbrand.csv
✅ Success: data/csv_dup_rows/20250222_051443_markus.herbrand.csv
💾 Saved updated accounts list


100%|██████████| 1348/1348 [00:00<00:00, 25027.43it/s]


Imported Zeeschuimer Data. Saved export to data/csv_dup_rows/20250222_051443_christianhirtemdb.csv
✅ Success: data/csv_dup_rows/20250222_051443_christianhirtemdb.csv
💾 Saved updated accounts list
⚠️ Skipping row 537: Missing Export_Path or IG_handle
⚠️ Skipping row 538: Missing Export_Path or IG_handle
⚠️ Skipping row 539: Missing Export_Path or IG_handle
⚠️ Skipping row 540: Missing Export_Path or IG_handle
⚠️ Skipping row 541: Missing Export_Path or IG_handle
⚠️ Skipping row 542: Missing Export_Path or IG_handle
⚠️ Skipping row 543: Missing Export_Path or IG_handle
⚠️ Skipping row 544: Missing Export_Path or IG_handle
⚠️ Skipping row 545: Missing Export_Path or IG_handle
⚠️ Skipping row 546: Missing Export_Path or IG_handle
⚠️ Skipping row 547: Missing Export_Path or IG_handle
⚠️ Skipping row 548: Missing Export_Path or IG_handle
⚠️ Skipping row 549: Missing Export_Path or IG_handle
⚠️ Skipping row 550: Missing Export_Path or IG_handle
⚠️ Skipping row 551: Missing Export_Path or IG_h

# Deduplicate all CSVs while keeping all unique infos

In [11]:
import os

In [12]:
# def finish_csvs_and_create_global():
#     accounts_with_csvs = pd.read_csv("data/csv_dup_rows/accounts_dup_rows.csv")

#     global_posts_path = "data/global_posts.csv"

#     global_posts_df = pd.DataFrame()

#     if os.path.exists(global_posts_path):
#         global_posts_df = pd.read_csv(global_posts_path)


#     for idx, row in accounts_with_csvs.iterrows():
#         handle = row["IG_handle"]
#         timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#         export_filename = f"data/post_csvs/{timestamp}_{handle}.csv"   
        
        
#         csv_path = row["CSV_Path"]
#         post_df = pd.read_csv(csv_path)
#         agg_rules = {col: "first" for col in post_df.columns}
#         agg_rules["image_url"] = list   
#         agg_rules["media_url"] = list
#         post_df = post_df.groupby("id", as_index=False).agg(agg_rules)

#         post_df.to_csv(export_filename, index=False)
#         accounts_with_csvs.at[idx, "CSV_Path"] = export_filename
#         accounts_with_csvs.to_csv("data/accounts_analysis.csv", index=False)
#         print(f"Deduplicated and stored to {export_filename}")

#         global_posts_df = pd.concat([global_posts_df, post_df], ignore_index=True)
#         global_posts_df.to_csv(global_posts_path, index=False)
#         print(f"Appended {export_filename} to global posts")
    
    
#     print(f"All deduplications done and all post file paths saved to data/post_csvs/... \nAll posts stored in global file {global_posts_path}")
    

        
    

In [14]:
def finish_csvs_and_create_global():
    accounts_with_csvs = pd.read_csv("data/csv_dup_rows/accounts_dup_rows.csv")
    global_posts_path = "data/global_posts.csv"
    
    # Initialize or load global posts
    global_posts_df = pd.read_csv(global_posts_path) if Path(global_posts_path).exists() else pd.DataFrame()

    for idx, row in accounts_with_csvs.iterrows():
        try:
            # Validate required fields
            if pd.isna(row.get("CSV_Path")) or pd.isna(row.get("IG_handle")):
                print(f"⚠️ Skipping row {idx}: Missing CSV_Path or IG_handle")
                continue
                
            csv_path = row["CSV_Path"]
            handle = row["IG_handle"]

            # Validate path exists and is CSV
            if not isinstance(csv_path, str) or not csv_path.endswith('.csv'):
                print(f"🔄 Invalid path format for {handle}: {csv_path}")
                continue
                
            if not Path(csv_path).exists():
                print(f"📂 Missing CSV for {handle} at {csv_path}")
                continue

            # Read with validation
            try:
                post_df = pd.read_csv(csv_path)
                if post_df.empty:
                    print(f"📭 Empty CSV for {handle}")
                    continue
            except Exception as e:
                print(f"📖 Read error for {handle}: {str(e)}")
                continue

            # Safe aggregation
            try:
                agg_rules = {col: "first" for col in post_df.columns}
                agg_rules.update({"image_url": list, "media_url": list})
                post_df = post_df.groupby("id", as_index=False).agg(agg_rules)
            except KeyError as ke:
                print(f"🔑 Missing column {str(ke)} in {handle}")
                continue

            # Generate new path
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            export_filename = f"data/post_csvs/{timestamp}_{handle.strip()}.csv"
            
            # Save processed file
            post_df.to_csv(export_filename, index=False)
            accounts_with_csvs.at[idx, "CSV_Path"] = export_filename
            
            # Update global dataset
            global_posts_df = pd.concat([global_posts_df, post_df], ignore_index=True)
            print(f"✅ Processed {handle}")

        except Exception as e:
            print(f"🔥 Critical error with {handle}: {str(e)}")
            traceback.print_exc()

    # Save outputs only if successful
    global_posts_df.to_csv(global_posts_path, index=False)
    accounts_with_csvs.to_csv("data/accounts_analysis.csv", index=False)
    print(f"🏁 Finalized global dataset with {len(global_posts_df)} posts")

In [15]:
finish_csvs_and_create_global()

✅ Processed bundeskanzler
✅ Processed abaerbock
✅ Processed minister.habeck
✅ Processed dorobaer
⚠️ Skipping row 4: Missing CSV_Path or IG_handle
⚠️ Skipping row 5: Missing CSV_Path or IG_handle
⚠️ Skipping row 6: Missing CSV_Path or IG_handle
✅ Processed karl_lauterbach_mdb
⚠️ Skipping row 8: Missing CSV_Path or IG_handle
⚠️ Skipping row 9: Missing CSV_Path or IG_handle
✅ Processed sahra_wagenknecht
⚠️ Skipping row 11: Missing CSV_Path or IG_handle
⚠️ Skipping row 12: Missing CSV_Path or IG_handle
✅ Processed amtsbergluise
✅ Processed olafscholz
⚠️ Skipping row 15: Missing CSV_Path or IG_handle
✅ Processed robert.habeck
✅ Processed rogerbeckamp
✅ Processed gregorgysi
⚠️ Skipping row 19: Missing CSV_Path or IG_handle
✅ Processed saskiaesken
⚠️ Skipping row 21: Missing CSV_Path or IG_handle
⚠️ Skipping row 22: Missing CSV_Path or IG_handle
⚠️ Skipping row 23: Missing CSV_Path or IG_handle
✅ Processed cem.oezdemir
✅ Processed ralph_brinkhaus
⚠️ Skipping row 26: Missing CSV_Path or IG_han

In [17]:
global_posts_df = pd.read_csv("data/global_posts.csv")
global_posts_df.head()

/var/folders/dg/8_1dqh0j1sq6n3btfr7g__pc0000gn/T/ipykernel_28488/88907028.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  global_posts_df = pd.read_csv("data/global_posts.csv")


,Unnamed: 0,id,thread_id,parent_id,body,author,author_fullname,author_avatar_url,timestamp,type,url,image_url,media_url,hashtags,num_likes,num_comments,num_media,location_name,location_latlong,location_city,unix_timestamp,media_n,media_filename
0,452,C--xuvlogWQ,C--xuvlogWQ,C--xuvlogWQ,"Wenn jemand in Schwierigkeiten steckt, dann pa...",bundeskanzler,Olaf Scholz,https://scontent-fra3-2.cdninstagram.com/v/t51...,2024-08-22 19:42:13,video,https://www.instagram.com/p/C--xuvlogWQ,['https://scontent-fra5-1.cdninstagram.com/v/t...,['https://scontent-fra3-2.cdninstagram.com/o1/...,"MeyerWerft,Papenburg,Werft,Schiffsbau,Bundeska...",3809.0,646.0,1.0,NaN,NaN,NaN,1.724349e+09,0.0,C--xuvlogWQ
1,469,C-2Z4wfouLq,C-2Z4wfouLq,C-2Z4wfouLq,Zusammenhalt und ein gutes Miteinander – dafür...,bundeskanzler,Olaf Scholz,https://scontent-fra3-2.cdninstagram.com/v/t51...,2024-08-19 13:37:06,mixed,https://www.instagram.com/p/C-2Z4wfouLq,['https://scontent-fra3-2.cdninstagram.com/v/t...,['https://scontent-fra3-2.cdninstagram.com/v/t...,"Stoppelmarkt,Vechta,Jahrmarkt,Bundeskanzler,Ka...",7956.0,363.0,7.0,NaN,NaN,NaN,1.724067e+09,0.0,C-2Z4wfouLq_0
2,464,C-3A9ERomNm,C-3A9ERomNm,C-3A9ERomNm,Aida und Mouhamed sind als Teenager aus Äthiop...,bundeskanzler,Olaf Scholz,https://scontent-fra3-2.cdninstagram.com/v/t51...,2024-08-19 19:18:28,photo,https://www.instagram.com/p/C-3A9ERomNm,['https://scontent-fra5-1.cdninstagram.com/v/t...,['https://scontent-fra5-1.cdninstagram.com/v/t...,"Einbürgerung,Einwanderungsgesetz,Bremen,Bundes...",10972.0,611.0,5.0,NaN,NaN,NaN,1.724088e+09,0.0,C-3A9ERomNm_0
3,463,C-4WayMIhLP,C-4WayMIhLP,C-4WayMIhLP,"Kilmerstuten, frisch gezapftes Bier, Karussell...",bundeskanzler,Olaf Scholz,https://scontent-fra3-2.cdninstagram.com/v/t51...,2024-08-20 07:51:16,video,https://www.instagram.com/p/C-4WayMIhLP,['https://scontent-fra3-2.cdninstagram.com/v/t...,['https://scontent-fra3-2.cdninstagram.com/o1/...,"Stoppelmarkt,Vechta,Bundeskanzler,Kanzler,Olaf...",6561.0,403.0,1.0,NaN,NaN,NaN,1.724133e+09,0.0,C-4WayMIhLP
4,459,C-5MEWio9OQ,C-5MEWio9OQ,C-5MEWio9OQ,Schon heute stecken in einem E-Auto mindestens...,bundeskanzler,Olaf Scholz,https://scontent-fra3-2.cdninstagram.com/v/t51...,2024-08-20 15:34:04,mixed,https://www.instagram.com/p/C-5MEWio9OQ,['https://scontent-fra5-1.cdninstagram.com/v/t...,['https://scontent-fra5-1.cdninstagram.com/v/t...,"InnovateTogether,TSMC,Halbleiter,Semiconductor...",2843.0,284.0,4.0,NaN,NaN,NaN,1.724161e+09,0.0,C-5MEWio9OQ_0


In [28]:
len(global_posts_df["body"])

532882

In [31]:
word_count = global_posts_df["body"].str.split().str.len().sum()
char_count = global_posts_df["body"].str.len().sum()
f"{word_count} words consisting of {char_count} characters"

'39756671.0 words consisting of 297790588.0 characters'

In [20]:
accounts_df = pd.read_csv("data/accounts_analysis.csv")

In [22]:
accounts_df["ZS_count"].sum()

560942

In [25]:
accounts_df["IG_count"].sum()

583604

In [23]:
accounts_df["ZS_count"].max()

10557

In [26]:
accounts_df["ZS_count"].mean()

748.9212283044059

In [27]:
accounts_df["ZS_count"].median()

610.0